In [8]:
import numpy as np
from scipy.sparse import kron, eye
from scipy.sparse.linalg import eigs

# Define the parameters of the Heisenberg model
# Here we suppose the external magnetic field is zero
J = 1  # Exchange interaction energy

# Define the size of the system (a 2 × L lattice with two L-length chains)
L = 6

# Construct the Hamiltonian matrix, H1: intra-chain interaction & H2: inter-chain interaction
H1 = np.zeros((2**(2*L), 2**(2*L)), dtype=np.complex128)
H2 = np.zeros((2**(2*L), 2**(2*L)), dtype=np.complex128)
H3 = np.zeros((2**(2*L), 2**(2*L)), dtype=np.complex128)

for i in range(1, L, 2):
    # Define the exchange interaction term between adjacent spins (1D chain)
    SxSx1 = kron(kron(kron(eye(2**(i-1)), kron([[0, 1], [1, 0]], [[0, 1], [1, 0]])), eye(2**(L-i-1))), eye(2**L))
    SySy1 = kron(kron(kron(eye(2**(i-1)), kron([[0, -1j], [1j, 0]], [[0, -1j], [1j, 0]])), eye(2**(L-i-1))), eye(2**L))
    SzSz1 = kron(kron(kron(eye(2**(i-1)), kron([[1, 0], [0, -1]], [[1, 0], [0, -1]])), eye(2**(L-i-1))), eye(2**L))
    # Add the exchange interaction term to the Hamiltonian matrix
    H1 -= J * (SxSx1 + SySy1 + SzSz1)
    # Define the exchange interaction term between adjacent spins (1D chain)
    SxSx2 = kron(eye(2**L), kron(kron(eye(2**(i-1)), kron([[0, 1], [1, 0]], [[0, 1], [1, 0]])), eye(2**(L-i-1))))
    SySy2 = kron(eye(2**L), kron(kron(eye(2**(i-1)), kron([[0, -1j], [1j, 0]], [[0, -1j], [1j, 0]])), eye(2**(L-i-1))))
    SzSz2 = kron(eye(2**L), kron(kron(eye(2**(i-1)), kron([[1, 0], [0, -1]], [[1, 0], [0, -1]])), eye(2**(L-i-1))))
    # Add the exchange interaction term to the Hamiltonian matrix
    H1 -= J * (SxSx2 + SySy2 + SzSz2)

for i in range(2, L, 2):
    # Define the exchange interaction term between adjacent spins (1D chain)
    SxSx1 = kron(kron(kron(eye(2**(i-1)), kron([[0, 1], [1, 0]], [[0, 1], [1, 0]])), eye(2**(L-i-1))), eye(2**L))
    SySy1 = kron(kron(kron(eye(2**(i-1)), kron([[0, -1j], [1j, 0]], [[0, -1j], [1j, 0]])), eye(2**(L-i-1))), eye(2**L))
    SzSz1 = kron(kron(kron(eye(2**(i-1)), kron([[1, 0], [0, -1]], [[1, 0], [0, -1]])), eye(2**(L-i-1))), eye(2**L))
    # Add the exchange interaction term to the Hamiltonian matrix
    H2 -= J * (SxSx1 + SySy1 + SzSz1)
    # Define the exchange interaction term between adjacent spins (1D chain)
    SxSx2 = kron(eye(2**L), kron(kron(eye(2**(i-1)), kron([[0, 1], [1, 0]], [[0, 1], [1, 0]])), eye(2**(L-i-1))))
    SySy2 = kron(eye(2**L), kron(kron(eye(2**(i-1)), kron([[0, -1j], [1j, 0]], [[0, -1j], [1j, 0]])), eye(2**(L-i-1))))
    SzSz2 = kron(eye(2**L), kron(kron(eye(2**(i-1)), kron([[1, 0], [0, -1]], [[1, 0], [0, -1]])), eye(2**(L-i-1))))
    # Add the exchange interaction term to the Hamiltonian matrix
    H2 -= J * (SxSx2 + SySy2 + SzSz2)

for i in range(1,L+1):
    # Define the exchange interaction term between adjacent spins (inter-chain)
    SxSx3 = kron(kron(kron(eye(2**(i-1)), [[0, 1], [1, 0]]), eye(2**(L-i))), kron(kron(eye(2**(i-1)), [[0, 1], [1, 0]]), eye(2**(L-i))))
    SySy3 = kron(kron(kron(eye(2**(i-1)), [[0, -1j], [1j, 0]]), eye(2**(L-i))), kron(kron(eye(2**(i-1)), [[0, -1j], [1j, 0]]), eye(2**(L-i))))
    SzSz3 = kron(kron(kron(eye(2**(i-1)), [[0, 1], [1, 0]]), eye(2**(L-i))), kron(kron(eye(2**(i-1)), [[0, 1], [1, 0]]), eye(2**(L-i))))
    # Add the exchange interaction term to the Hamiltonian matrix
    H3 -= J * (SxSx3 + SySy3 + SzSz3)

# Shift the Hamiltonian by the ground state energy
# Lanczos Method for sparse matrix. By default, the output is sorted in ascending order of absolute value
energy1, _ = eigs(H1, k=4096, which='SR')
energy2, _ = eigs(H2, k=4096, which='SR')
energy3, _ = eigs(H3, k=4096, which='SR')
ground_energy1 = np.min(energy1)
ground_energy2 = np.min(energy2)
ground_energy3 = np.min(energy3)

# Floor function removed
H1_shift = H1 - ground_energy1 * eye(2**(2*L))
H2_shift = H2 - ground_energy2 * eye(2**(2*L))
H3_shift = H3 - ground_energy3 * eye(2**(2*L))

# Derive the global Hamiltonian
H = H1 + H2 + H3
H_shift = H1_shift + H2_shift + H3_shift
energy, states = eigs(H_shift, k=4096)

/opt/anaconda3/envs/AntiEntropy/lib/python3.11/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:1272: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  warnings.warn("k >= N - 1 for N * N square matrix. "


In [37]:
energy1[1]

(-5.999999999999999+0j)